In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
import pandas as pd
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans 
import matplotlib.pyplot as plt
import numpy as np

In [3]:
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(
    client_id="f427c27f15b8441a8122e307a960d011", client_secret="3b924a72bef446f2ad8ba21032c5bcbb"))

Global Top 50

In [9]:
playlist = spotify.user_playlist_tracks(user="spotify",playlist_id='spotify:playlist:37i9dQZF1EpMHnOLKP4IRH')

In [23]:
artists_list

set()

In [5]:
#List of artists
artists_list =set()
for item in playlist['items']:
    for artist in item ['track']['artists']:
        artists_list.add(artist['uri'])

In [6]:
# From  artist to album
albums =[]
for artist in artists_list:
    albums.append(spotify.artist_albums(artist_id = artist))
    
albums_list = set()
for album in albums:
    for item in album['items']:
        albums_list.add(item['uri'])

In [37]:
#From album to song
songs =[]
for album in albums_list:
    songs.append(spotify.album_tracks(album_id=album))
    time.sleep(1)


songs_list=set()
for song in songs:
    for item in song['items']:
        songs_list.add(item['uri'])


In [38]:
len(songs_list)

4139

In [41]:
features_all=[]
for songs in songs_list:
    features_all.append(spotify.audio_features(songs))
    time.sleep(2)

ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

In [ ]:
#Saaaaaaaaave in csv!!!

In [ ]:
X = pd.DataFrame(features_all[0], columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'])

for i in range(1,len(features_all)):
    X = X.append(pd.DataFrame((features_all[i]), columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']),ignore_index=True)

In [ ]:
scaler = StandardScaler()

In [ ]:
X_scaled =scaler.fit_transform(X)
kmeans = KMeans(n_clusters=8, random_state=1234)  #cluster shoud be changed
kmeans.fit(X_scaled)
clusters = kmeans.predict(X_scaled)

In [ ]:
X["clusters"] = clusters

In [ ]:
X["id"] = X.index

# Recommendation from Spotify

In [ ]:
#Insert feature_input_song

In [ ]:
df_input_song = pd.DataFrame(features_input_song, columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'])

#get the scale and cluster
song_scaled = scaler.transform(df_input_song)
kmeans.predict(song_scaled)
clust = int(kmeans.predict(song_scaled))

#get the list of songs from the same cluster randomly
list1 = (X[X["clusters"] == clust]["id"].values).tolist()
new_song_uri = songs_list[random.choice(list1)]

print("Here is your recommendation: " ,spotify.track(new_song_uri)['name'], "from: ", spotify.track(new_song_uri)['artists'][0]['name'])